# 4.深入剖析PyTorch nn.Module源码

知乎上有pytorch源码解读系列，推荐一下[OpenMMLab](https://zhuanlan.zhihu.com/p/340453841)的专栏

B站up主deep_thoughts认为:
"这期视频是写PyTorch代码少走弯路的必看视频，源码角度去理解nn.Module模块的视频，nn.Module是PyTorch几乎所有模块的核心父类，看完本期视频，会对PyTorch代码有全新的认识，看过的都说收获很大并且是全b站唯一一个此类视频 "接下来是笔记：

`Module`是所有神经网络模块的基类。 我们自己的模型也应该继承这个类。 模块还可以包含其他模块，允许将它们嵌套在树结构中。

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        '''
        必须在对子类进行赋值之前对父类进行 __init__() 调用。
        '''
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5) #这里卷积层作为被包含的模块
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

## Module的方法：
1. add_module(name, module) 为模块增加子模块，通过'.'访问;
2. apply(fn) 函数作为参数，可以递归的应用到子模块中（可通过`.children()`返回的子模块），通常用于初始化;
   

In [4]:
import torch
@torch.no_grad()#修饰符，对下面的函数不需要计算梯度
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight,"\n")
net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weights)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True) 

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True) 

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

3. buffers(recurse=True) 每个参数都有附属的统计量，这边称作buffer，tensorflow称为槽变量
之后还会有介绍

In [ ]:
for buf in model.buffers():
    print(type(buf), buf.size())

4. eval()设置验证模式，与训练模式的`Dropout`,`BatchNorm`不同
5. load_state_dict(state_dict, strict=True)
state_dict存储模型的所有参数和buffer，加载模型时使用，从静态字典中加载.
6. requires_grad_(requires_grad=True)设置是否需要梯度更新，finetune时很有用.

这里插入说一下`state_dict`和模型保存


In [ ]:
#保存checkpoint
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),#保存参数和buffer，可以用于推理
            'optimizer_state_dict': optimizer.state_dict(),#优化器
            'loss': loss,
            ...
            }, PATH)
            
#加载checkpoint
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

## Module python源码解读

位置:torch\nn\modules\module.py 204行~end(1800行左右)
挑选用的比较多的函数，讲解原理

``1.__init__``

``2.register_buffer`` 
`  def register_buffer(self, name: str, tensor: Optional[Tensor], persistent: bool = True) -> None:`
向当前模块添加buffer变量
buffer默认是持久的，可以和参数一样保存下来
常用于BatchNorm里面running_mean这些统计量，自己实现：
```python
class BatchNorm(nn.Module):
    def __init__(self):
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_variance', torch.ones(num_features))
```

``3.register_parameter``
使用频率比`register_buffer`更高 
`def register_parameter(self, name: str, param: Optional[Parameter]) -> None:` 
[Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html)类型是tensor类型的继承，模块中定义参数都要写成parameter的实例
在module中写一个Parameter实例会自动添加到Patameters的列表中，无需手动添加，实例：
```python
class GaussianModel(nn.Module):
    def __init__(self):
        super(GaussianModel, self).__init__()

        self.register_parameter('mean',nn.Parameter(torch.zeors(1),requires_grad = True))

        self.pdf = torch.distributions.Normal(self.state_dict()['means'],torch.tensor([1.0]))

    def forward(self, x):
        return -self.pdf.log_prob(x)

model = GaussianModel()
```
``4.get_submodule`` 通过字符串获得子模块

``5.get_parameter`` 通过字符串获得参数

``6.get_buffer`` 通过字符串获得buffer

``7.apply`` 将方法递归应用到每个子模块 


